In [1]:
#install BeautifulSoup if needed
pip install beautifulsoup4

In [2]:
#import necessary modules
import requests
from bs4 import BeautifulSoup
from urllib.request import urlopen 
import urllib
import os
from os import listdir
from os.path import isfile, join
import xml.etree.ElementTree as ET
import pandas as pd
import csv

In [15]:
#https://www.codeproject.com/Articles/1227268/Accessing-Financial-Reports-in-the-EDGAR-Database
#First figure out the CIK number for the company you are looking for. update accordingly

urlstart="https://www.sec.gov/cgi-bin/browse-edgar?action=getcompany"
CIK="0001765880"
reporttype="ABS-EE"
#key is going to be used in BS to find the links to all the links
key='Archives/edgar/data'
#key1 is going to be what report name you want to filter on
key1="exh102"
#this is used in a few places to star the link to the sec's site
url2="https://sec.gov"

#let's add the count=100 to make sure we get all the filings 
fullurl=urlstart+"&CIK="+CIK+"&type="+reporttype+"&count=100"
#print(fullurl)

In [16]:
#import the url to make sure it works. Code of 200 means its good to go. Codes that start with 4 or 5 typically mean there was an error
page = requests.get(fullurl)
page

<Response [200]>

In [17]:
#get the html using beautiful soup
soup = BeautifulSoup(page.content, 'html.parser')
#print(soup.prettify())

In [18]:
#create a list of urls that contain archives/edgar/data
urls=[]
html = urlopen(fullurl)
soup = BeautifulSoup(html.read(), 'lxml')
links = []
for link in soup.find_all('a'):
    if key in str(link):
        urls.append(link.get('href'))
#print(urls)

In [19]:
#from the list of urls created in the last step, visit each and get a list of urls that contain the link to 
#the actual xml files. filter on 'xh102'
urls1=[]
for url in urls:
    newurl=url2+url
    page1 = requests.get(newurl)
    soup1 = BeautifulSoup(page1.content, 'html.parser')
    for link in soup1.find_all('a'):
        if key1 in str(link):
            urls1.append(link.get('href'))
#print(urls1)


In [20]:
#save the xml files down. this code works. write the loop to save all files down

#Create a subfolder in 'SEC' labelled with the CIK#
if not os.path.exists('sec/'+CIK):
    os.makedirs('sec/'+CIK)
#get a list of files in that directory
onlyfiles = [os.path.splitext(f)[0] for f in listdir('sec/'+CIK) if isfile(join('sec/'+CIK, f))]

#if the filename you are referencing is NOT in the directory already, download
for url in urls1:
    path=url2+url
    filenamefull=path[path.rfind("/")+1:]
    filename=os.path.splitext(path[path.rfind("/")+1:])[0]
    if filename not in onlyfiles:
        urllib.request.urlretrieve(path, "sec/"+CIK+"/"+filenamefull)

In [21]:
#xml files are real big. Let's go through the folder and convert the data to CSV files. CSV files are ~5-8x smaller than XML
#a function that will convert an XML to a dataframe
def parse_XML(xml_file, df_cols): 
    """Parse the input XML file and store the result in a pandas 
    DataFrame with the given columns. 
    """
    
    xtree = ET.parse(xml_file)
    xroot = xtree.getroot()
    rows = []
    
    for node in xroot: 
        res = []
        for el in df_cols[0:]: 
            if node is not None and node.find(el, namespaces) is not None:
                res.append(node.find(el, namespaces).text)
            else: 
                res.append(None)
        rows.append({df_cols[i]: res[i] 
                     for i, _ in enumerate(df_cols)})
    
    out_df = pd.DataFrame(rows, columns=df_cols)
        
    return out_df

In [22]:
columns=[]
with open("ABS EE Columns.txt", 'r') as inputfile:
    columns=inputfile.read().split(",")
#print(columns)

In [23]:
#get a list of xml files to convert
xmlfiles=[file for file in os.listdir('sec/'+CIK) if file.endswith(".xml")]
print(xmlfiles)

['exh1025940092020.xml']


In [24]:
#go through each file, turn into dataframe, export to CSV, delete XML file
for file in xmlfiles:
    #Uses a list comprehension and element tree's iterparse function to create a dictionary containing the namespace prefix and it's uri
    #The underscore is utilized to remove the "start-ns" output from the list.
    namespaces = {node[0]: node[1] for _, node in ET.iterparse('sec/'+CIK+'/'+file, events=['start-ns'])}
#Iterates through the newly created namespace list registering each one.
    for key, value in namespaces.items():
        ET.register_namespace(key, value)
    #print(namespaces)
    a=parse_XML('sec/'+CIK+'/'+file,columns)
    a.to_csv('sec/'+CIK+'/'+file.rstrip(".xml")+'.csv', index=False)
    os.remove('sec/'+CIK+'/'+file)